In [2]:
import torch
from torchvision import models
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST
import numpy as np

from tqdm import tqdm

In [3]:
torch.cuda.is_available()

False

In [ ]:
torch.cuda.get_device_name()

In [20]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cpu')

In [ ]:
def get_model():
    # Загрузка модели ResNet50
    model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

    # Убираем последний полносвязный слой (FC)
    model.fc = torch.nn.Identity()

    # Выключаем вычисление градиентов и включаем режим оценки
    for param in model.parameters():
        param.requires_grad = False
    model.eval()

    model.to()
    return model

In [12]:
# Трансформация изображений
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),  # Теперь делаем изображение трёхканальным
    transforms.ToTensor(),
])

# Загрузка FashionMNIST
dataset = FashionMNIST(root="./data", train=True, download=True, transform=transform)

In [13]:
dataloader = DataLoader(dataset, batch_size=64, shuffle=False)

In [10]:
model = get_model()

In [18]:
# Извлекаем признаки
features_list = []
labels_list = []

with torch.no_grad():
    for images, labels in tqdm(dataloader):
        outputs = model(images)
        features_list.append(outputs.cpu().numpy())
        labels_list.append(labels.numpy())

# Обрабатываем полученные признаки и метки
all_features = np.concatenate(features_list, axis=0)
all_labels = np.concatenate(labels_list, axis=0)

  0%|          | 0/938 [00:00<?, ?it/s]

  0%|          | 3/938 [00:22<1:56:07,  7.45s/it]


KeyboardInterrupt: 

In [ ]:
print("Размер массива признаков:", all_features.shape)
print("Количество классов:", len(np.unique(all_labels)))